# Stage 1: Data ingestion and Data exploration

The objective is to take the first steps in the process of data exploration using SQL, Python3 with Numpy, Pandas and Matplotlib libraries.

I'll focus on analyzing some aspects of the real estate market of the City of Buenos Aires but for that I must first acquire the dataset from a BigQuery database.

# BigQuery data ingestion

To access the database I use an username provided by the companyto access the BigQuery database they have in Gcloud platform.
Once accessed the BigQuery database I do an exploratory analysis of the database composition.


## The database: Datapoints contain publication of properties for sale or rent with the property specifications and price. This database contains data from many Latam countries each one with their own table.


### Raw features of the ar_properties table are:

**id**: property id  
**start_date:** date of first publication  
**end_date:** end of publication  
**created_on:** date of publication creation  
**lat:** latitude of property location  
**lon:** longitude of property location  
**l1:** country  
**l2:** Province  
**l3:** City  
**l4:** Neighborhood  
**l5:** other location details  
**l6:** other location details  
**rooms:** how many total rooms  
**bedrooms:** how many bedrooms  
**bathrooms:** how many bathrooms  
**surface_total:** surface total in square meters  
**surface_covered:** surface covered in square meters (surface without counting balcony, courtyards and the likes)  
**price:** total price  
**currency:** currency in USD or ARS  
**price_period:** payment pediod if renting  
**title:** title of the publication  
**description:** description of the property  
**property_type:** property type  
**operation_type:** for sale or for rent  

For this project I will only use data from Argentina, more specificly from apartments in Buenos Aires City. For this purpouse I'm going to make a query filtering data to get only apartments from Buenos Aires City.


### The query:

Download the dataset as a .csv and load onto this notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import pandas_profiling

Also its important to import the rest of the libraries I'm going to use throughout this stage. Numpy to handle math functions and Pandas to handle Dataframes and Series. Then Matplotlib and Seaborn for visualization of the data. And pandas_profiling for a first glimpse of the data.  
Then some settings to make sure our data is easier to analyse

In [2]:
# I make sure all ploting happens inside the notebook
%matplotlib inline
# then I set the figures size for all my ploting
plt.rcParams['figure.figsize'] = (12,8)
# then I set the max number of columns Pandas will show if I call the dataset
pd.set_option('max_columns', 120)
pd.set_option('display.float_format', lambda x: '%.2f' % x) #toggle scientic notation

## Data Exploration

In [3]:
df = pd.read_csv('./Dataset.csv') #importing the dataset

In [4]:
report = pandas_profiling.ProfileReport(df)

C:\Users\ppmat\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 17).')
  correlation_name=correlation_name, error=error


In [5]:
report


# Handling the warnings of the profiling.

### Following the profiling report, many features won't give me much information.  I'll remove those columns to keep valuable features only.  
Features to be droped:
- Created_on: all dates are from 2019 and that's ok, no need to keep this column.  
- Features with constant values: l1, l2, property_type and operation_type. These values are constant because have been used as a filter in BigQuery. 
- l4, l5 and l6 have no data.  

In [6]:
df = df[['lat', 'lon', 'l3', 'rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'price', 'currency','ars_price']]


### It seems the Dataset contains many duplicate rows. Lets find out which ones and removed them if needed.

In [7]:
mask_dup = df.duplicated(keep=False)
duplicate_rows = df.loc[mask_dup, :]
duplicate_rows.head()

lat    lon            l3  rooms  bedrooms  bathrooms  surface_total  \
0  -34.58 -58.43       Palermo   1.00      0.00       1.00          21.00   
1  -34.65 -58.50     Mataderos   1.00      0.00       2.00          35.00   
2  -34.65 -58.50     Mataderos   1.00      0.00       2.00          35.00   
9  -34.58 -58.43       Palermo   1.00       nan       1.00          45.00   
11 -34.60 -58.44  Villa Crespo   1.00       nan       1.00         109.00   

    surface_covered    price currency  ars_price  
0               nan 75000.00      USD 4875000.00  
1             35.00 80000.00      USD 5200000.00  
2             35.00 80000.00      USD 5200000.00  
9             33.00     0.00      NaN       0.00  
11           109.00     0.00      NaN       0.00

Now I'll remove duplicates keeping the first row of each group.

In [8]:
df.drop_duplicates(keep='first', inplace=True)

Then I'll drop rows with missing data in important features which can't be calculated through any means, like total rooms or surface_covered or price

In [9]:
df.dropna(subset = ['rooms', 'surface_covered','price'], inplace=True)

Then I'll drop instances with value 0 in their price feature

In [10]:
df = df[df['price']!=0]

### ars_price feature:  some outliers and unreal values.


In [26]:
df.sort_values(by='ars_price', ascending=False)

lat    lon                 l3  rooms  bedrooms  bathrooms  \
688   -34.59 -58.40           Recoleta   8.00      5.00       5.00   
26810 -34.59 -58.39           Recoleta  10.00      4.00       3.00   
6301  -34.61 -58.45          Caballito   1.00       nan       1.00   
41083 -34.57 -58.42            Palermo   4.00       nan       4.00   
7863  -34.60 -58.38        San Nicolás   3.00       nan       1.00   
15160 -34.58 -58.43            Palermo   5.00      3.00       4.00   
56695 -34.58 -58.41            Palermo   5.00      4.00       3.00   
20438 -34.58 -58.40            Palermo   5.00       nan       5.00   
39243 -34.58 -58.41            Palermo   6.00      4.00       5.00   
5421  -34.58 -58.41            Palermo   5.00       nan       4.00   
26437 -34.59 -58.39           Recoleta   7.00      4.00       4.00   
14927 -34.58 -58.41            Palermo   4.00      3.00       2.00   
5194  -34.58 -58.40            Palermo   4.00      3.00       3.00   
12503 -34.59 -58.38           Recoleta   5.00      3.00       5.00   
55532 -34.58 -58.40            Palermo   4.00       nan       3.00   
36376 -34.58 -58.40            Palermo   4.00       nan       3.00   
15174 -34.55 -58.44           Belgrano   5.00       nan       4.00   
1304     nan    nan           Recoleta   5.00      3.00       5.00   
36389 -34.58 -58.40            Palermo   5.00       nan       3.00   
39663 -34.58 -58.40            Palermo   4.00      3.00       3.00   
26424 -34.58 -58.40            Palermo   6.00      3.00       3.00   
4029  -34.61 -58.36      Puerto Madero   6.00      5.00       6.00   
43306 -34.57 -58.44            Palermo   6.00       nan       6.00   
47649 -34.57 -58.42            Palermo   5.00       nan       3.00   
4104  -34.57 -58.44           Belgrano   6.00       nan       6.00   
25740 -34.58 -58.40            Palermo   5.00      4.00       4.00   
40832 -34.57 -58.44           Belgrano   5.00       nan       6.00   
40831 -34.57 -58.44           Belgrano   5.00       nan       6.00   
6047  -34.57 -58.44           Belgrano   5.00      3.00       4.00   
6029  -34.58 -58.40            Palermo   4.00      3.00       3.00   
...      ...    ...                ...    ...       ...        ...   
6804  -34.59 -58.38             Retiro   5.00      3.00       2.00   
8884  -34.62 -58.46             Flores   1.00       nan       1.00   
25514 -38.97 -68.33                NaN   4.00       nan       1.00   
3870     nan    nan      Puerto Madero   3.00      2.00       2.00   
17660 -34.64 -58.45   Parque Chacabuco   1.00      1.00       1.00   
12071 -34.59 -58.38             Retiro   3.00      2.00       2.00   
47881 -34.59 -58.39           Recoleta   3.00      2.00       2.00   
55597 -34.58 -58.40            Palermo   5.00      4.00       4.00   
3948  -34.61 -58.36      Puerto Madero   3.00      2.00       3.00   
54867 -34.58 -58.48      Villa Urquiza   3.00       nan       2.00   
2573  -34.61 -58.39           Congreso   7.00      5.00       1.00   
43445 -34.60 -58.42       Barrio Norte   2.00      1.00       1.00   
38697 -34.57 -58.45           Belgrano   2.00      1.00       2.00   
13514 -34.58 -58.49      Villa Urquiza   4.00       nan       2.00   
4034  -34.55 -58.46           Belgrano   3.00       nan        nan   
16320 -34.61 -58.44          Caballito   4.00       nan       2.00   
18079 -34.57 -58.44        Las Cañitas   3.00       nan       3.00   
35168 -34.57 -58.44           Belgrano   3.00       nan       1.00   
33552 -34.58 -58.45         Colegiales   2.00      1.00       1.00   
54737 -34.63 -58.47             Flores   4.00      3.00       1.00   
43226 -34.57 -58.45           Belgrano   4.00      3.00       2.00   
15292 -34.60 -58.43       Villa Crespo   3.00      2.00       2.00   
11290 -34.62 -58.45          Caballito   3.00      2.00       2.00   
13515 -34.61 -58.42  Parque Centenario   4.00      3.00       1.00   
18080 -34.57 -58.44        Las Cañitas   3.00       nan       3.00   
46935 -34.59 -58.40

An outlier for this feature is it's maximun value. And wrongly imputed currency in the min values of the feature. 
There are no apartments within those lower range prices, so I'm assuming the correct imput for the currency us USD and not ARS. 


Instances with false data like 12121212 or all 6s seem manually imputed as a filler. 

In [24]:
df.drop([8878, 24709], inplace=True) 

USD prices below 12000 are rare in Buenos Aires housing market and tend to be an imputing error. Better to drop them to reduce noice from the dataset. 

In [28]:
mask_price_drop = df['price']<12000
df.drop(df.index[mask_price_drop], inplace=True ) 

Same as before but in ARS (USD12000*65=780000)

In [29]:
mask_price_drop_ars= df['ars_price']<780000
df.drop(df.index[mask_price_drop_ars], inplace=True) 

Small apartments cant cost that high so in those cases the currency was wrongly imputed. 

In [31]:
mask_surfaceprice = (df['surface_total'] <100) & (df['currency'] == 'USD') & (df['price'] >3000000)
df.loc[mask_surfaceprice,'currency']='ARS' 

After replacing some USD values by ARS values I had to recalculate feature ars_price

In [32]:
df['ars_price'] = df.apply(lambda row: row['price']*65 if row['currency'] == 'USD' else row['price'], axis=1)


In [33]:
df.sort_values(by='ars_price', ascending=False)


lat    lon                l3  rooms  bedrooms  bathrooms  \
688   -34.59 -58.40          Recoleta   8.00      5.00       5.00   
26810 -34.59 -58.39          Recoleta  10.00      4.00       3.00   
41083 -34.57 -58.42           Palermo   4.00       nan       4.00   
56695 -34.58 -58.41           Palermo   5.00      4.00       3.00   
15160 -34.58 -58.43           Palermo   5.00      3.00       4.00   
20438 -34.58 -58.40           Palermo   5.00       nan       5.00   
39243 -34.58 -58.41           Palermo   6.00      4.00       5.00   
26437 -34.59 -58.39          Recoleta   7.00      4.00       4.00   
5421  -34.58 -58.41           Palermo   5.00       nan       4.00   
14927 -34.58 -58.41           Palermo   4.00      3.00       2.00   
5194  -34.58 -58.40           Palermo   4.00      3.00       3.00   
12503 -34.59 -58.38          Recoleta   5.00      3.00       5.00   
55532 -34.58 -58.40           Palermo   4.00       nan       3.00   
36376 -34.58 -58.40           Palermo   4.00       nan       3.00   
1304     nan    nan          Recoleta   5.00      3.00       5.00   
36389 -34.58 -58.40           Palermo   5.00       nan       3.00   
15174 -34.55 -58.44          Belgrano   5.00       nan       4.00   
39663 -34.58 -58.40           Palermo   4.00      3.00       3.00   
4029  -34.61 -58.36     Puerto Madero   6.00      5.00       6.00   
26424 -34.58 -58.40           Palermo   6.00      3.00       3.00   
40831 -34.57 -58.44          Belgrano   5.00       nan       6.00   
15209 -34.55 -58.44             Nuñez   6.00      4.00       4.00   
43306 -34.57 -58.44           Palermo   6.00       nan       6.00   
40832 -34.57 -58.44          Belgrano   5.00       nan       6.00   
6029  -34.58 -58.40           Palermo   4.00      3.00       3.00   
25740 -34.58 -58.40           Palermo   5.00      4.00       4.00   
47517 -34.58 -58.40           Palermo   4.00      3.00       3.00   
47649 -34.57 -58.42           Palermo   5.00       nan       3.00   
4104  -34.57 -58.44          Belgrano   6.00       nan       6.00   
6047  -34.57 -58.44          Belgrano   5.00      3.00       4.00   
...      ...    ...               ...    ...       ...        ...   
3814  -34.66 -58.47         Mataderos   1.00      1.00       1.00   
29898 -34.63 -58.38      Constitución   1.00      1.00       1.00   
5362  -34.61 -58.40              Once   1.00       nan       1.00   
5452  -34.61 -58.40         Balvanera   1.00       nan       1.00   
48816 -34.65 -58.42             Boedo   1.00      1.00       1.00   
48949 -34.65 -58.42  Parque Patricios   1.00      1.00       1.00   
48851 -34.65 -58.42           Pompeya   1.00      1.00       1.00   
1380  -34.65 -58.42           Pompeya   1.00      1.00       1.00   
1736  -34.65 -58.46            Flores   2.00      1.00       1.00   
26882 -34.61 -58.38       San Nicolás   1.00      1.00        nan   
24729 -34.61 -58.38         Balvanera   1.00      1.00        nan   
32612 -34.67 -58.44     Villa Soldati   5.00       nan       1.00   
49296 -34.68 -58.49      Villa Lugano   1.00      1.00       1.00   
57600 -34.64 -58.50            Flores   1.00       nan       1.00   
44725 -34.64 -58.42           Pompeya   1.00      1.00       1.00   
23198 -34.64 -58.42           Pompeya   1.00      1.00       1.00   
23227 -34.64 -58.42  Parque Patricios   1.00      1.00       1.00   
37793 -34.62 -58.44         Caballito   1.00      1.00        nan   
1909  -34.67 -58.44     Villa Soldati   3.00      2.00       1.00   
16371 -34.62 -58.45        Villa Luro   4.00       nan       2.00   
17243 -34.55 -58.47           Coghlan   4.00      3.00       1.00   
27825 -34.59 -58.41      Barrio Norte   4.00       nan       2.00   
50380 -34.62 -58.41         Caballito   1.00       nan       1.00   
4806  -34.67 -58.44     Villa Soldati   5.00       nan       1.00   
2622  -34.58 -58.45        Colegiales   1.00      1.00       1.00   
5358  -34.57 -58.44       Las Cañitas   5.00      4.00       3.00   
8791  -

In [34]:
df['ars_price'].describe()


count       36527.00
mean     16266780.08
std      19346532.24
min        780000.00
25%       7475000.00
50%      10582000.00
75%      17420000.00
max     377000000.00
Name: ars_price, dtype: float64

### Next feature to process is bathrooms with 5% missing data

In [35]:
bathrooms = df.loc[df['bathrooms'].isna(), :]
bathrooms

lat    lon                    l3  rooms  bedrooms  bathrooms  \
16    -34.60 -58.40                   NaN   3.00       nan        nan   
117   -34.60 -58.50             Agronomía   1.00       nan        nan   
160   -34.60 -58.50      Villa del Parque   1.00       nan        nan   
172   -34.60 -58.38                Retiro   2.00      1.00        nan   
174   -34.61 -58.42               Almagro   2.00      1.00        nan   
296   -34.60 -58.40              Recoleta   3.00      2.00        nan   
315   -34.61 -58.39           San Nicolás   3.00      2.00        nan   
398   -34.61 -58.42               Almagro   3.00       nan        nan   
449   -34.64 -58.50            Villa Luro   3.00       nan        nan   
451   -34.64 -58.49            Villa Luro   1.00       nan        nan   
566   -34.59 -58.44          Villa Crespo   2.00      1.00        nan   
575   -34.60 -58.49      Villa del Parque   2.00      1.00        nan   
626   -34.63 -58.45             Caballito   3.00      2.00        nan   
638   -34.62 -58.40         San Cristobal   3.00      2.00        nan   
694   -34.64 -58.42                 Boedo   3.00       nan        nan   
860   -34.58 -58.42               Palermo   1.00       nan        nan   
861   -34.60 -58.38               Palermo   1.00       nan        nan   
1079  -34.60 -58.38  Centro / Microcentro   2.00      1.00        nan   
1236  -34.59 -58.41               Palermo   4.00      3.00        nan   
1358  -34.62 -58.52          Villa Devoto   1.00       nan        nan   
1405  -34.63 -58.40      Parque Patricios   2.00      1.00        nan   
1504  -34.60 -58.40             Balvanera   1.00       nan        nan   
1534  -34.65 -58.37              Barracas   2.00      1.00        nan   
1549  -34.62 -58.44             Caballito   2.00       nan        nan   
1567  -34.63 -58.46                Flores   3.00       nan        nan   
1569  -34.60 -58.43               Almagro   3.00       nan        nan   
1590  -34.60 -58.44          Villa Crespo   3.00      2.00        nan   
1613  -34.63 -58.42                 Boedo   5.00      4.00        nan   
1662  -34.56 -58.46              Belgrano   3.00       nan        nan   
1672  -34.60 -58.40              Recoleta   1.00       nan        nan   
...      ...    ...                   ...    ...       ...        ...   
56764 -34.58 -58.46            Colegiales   1.00       nan        nan   
56802 -34.61 -58.44             Caballito   2.00       nan        nan   
56810 -34.60 -58.41          Barrio Norte   2.00       nan        nan   
56815 -34.62 -58.38          Constitución   2.00      1.00        nan   
56836 -34.59 -58.41               Palermo   3.00       nan        nan   
56840 -34.56 -58.44              Belgrano   3.00       nan        nan   
56857 -34.61 -58.42             Caballito   3.00       nan        nan   
56885 -34.63 -58.46                Flores   4.00      3.00        nan   
56895    nan    nan              Barracas   4.00      3.00        nan   
56898 -34.56 -58.44              Belgrano   4.00       nan        nan   
56906 -34.63 -58.45             Caballito   4.00       nan        nan   
56941 -34.57 -58.45              Belgrano   1.00       nan        nan   
56942 -34.57 -58.45              Belgrano   2.00       nan        nan   
56954 -34.62 -58.53            Villa Real   3.00       nan        nan   
57228 -34.62 -58.38             Monserrat   1.00       nan        nan   
57306 -34.62 -58.42               Almagro   2.00      1.00        nan   
57345 -34.60 -58.40              Recoleta   1.00      1.00        nan   
57381 -34.60 -58.45          Villa Crespo   1.00      1.00        nan   
57388 -34.60 -58.43          Villa Crespo   1.00      1.00        nan   
57393 -34.60 -58.45          Villa Crespo   1.00      1.00        nan   
57394 -34.60 -58.45          Villa Crespo   1.00      1.00        nan   
57402 -34.62 -58.36         Puerto Madero   1.00      1.00        nan   
57487 -34.60 -58.45          Villa Crespo   3.00      2.00        n

In [36]:
df['bathrooms'].describe()

count   35358.00
mean        1.47
std         0.82
min         1.00
25%         1.00
50%         1.00
75%         2.00
max        14.00
Name: bathrooms, dtype: float64

In [37]:
df['rooms'].describe()

count   36527.00
mean        2.62
std         1.26
min         1.00
25%         2.00
50%         3.00
75%         3.00
max        18.00
Name: rooms, dtype: float64

In [38]:
df.sort_values(by='bathrooms', ascending=False)

lat    lon                    l3  rooms  bedrooms  bathrooms  \
4927  -34.58 -58.45            Colegiales   1.00       nan      14.00   
7429     nan    nan              Recoleta   2.00      1.00      14.00   
7380  -34.60 -58.38  Centro / Microcentro   1.00       nan      14.00   
38043 -34.58 -58.41               Palermo   1.00       nan      14.00   
38155 -34.58 -58.42               Palermo   2.00      1.00      14.00   
7346  -34.56 -58.46              Belgrano   1.00       nan      14.00   
2953  -34.57 -58.45              Belgrano   1.00      1.00      12.00   
26194 -34.59 -58.43               Palermo   2.00       nan      11.00   
1180  -34.59 -58.43          Villa Crespo   3.00      2.00      11.00   
11227 -34.64 -58.53               Liniers  10.00     10.00      10.00   
56923 -34.65 -58.49          Villa Lugano   9.00       nan       9.00   
46164 -34.60 -58.39              Recoleta   7.00      6.00       9.00   
45777 -34.65 -58.47          Villa Lugano   9.00      9.00       9.00   
15251 -34.60 -58.40              Recoleta  15.00       nan       9.00   
15248 -34.61 -58.38             Monserrat   9.00       nan       9.00   
28424 -34.58 -58.40               Palermo   8.00      4.00       7.00   
44515 -34.56 -58.45              Belgrano   7.00      5.00       7.00   
21726 -34.62 -58.37             San Telmo  15.00     12.00       7.00   
2564     nan    nan         Puerto Madero   4.00       nan       6.00   
3118     nan    nan              Belgrano   5.00      4.00       6.00   
43306 -34.57 -58.44               Palermo   6.00       nan       6.00   
40518    nan    nan              Belgrano   5.00       nan       6.00   
40524    nan    nan              Belgrano   6.00       nan       6.00   
17134    nan    nan          Barrio Norte   6.00      5.00       6.00   
3120     nan    nan              Belgrano   5.00      4.00       6.00   
29209 -34.61 -58.36         Puerto Madero   5.00      4.00       6.00   
4029  -34.61 -58.36         Puerto Madero   6.00      5.00       6.00   
4085  -34.56 -58.45              Belgrano   6.00       nan       6.00   
17135 -34.59 -58.39              Recoleta   6.00      5.00       6.00   
6128  -34.56 -58.45              Belgrano   8.00      4.00       6.00   
...      ...    ...                   ...    ...       ...        ...   
56764 -34.58 -58.46            Colegiales   1.00       nan        nan   
56802 -34.61 -58.44             Caballito   2.00       nan        nan   
56810 -34.60 -58.41          Barrio Norte   2.00       nan        nan   
56815 -34.62 -58.38          Constitución   2.00      1.00        nan   
56836 -34.59 -58.41               Palermo   3.00       nan        nan   
56840 -34.56 -58.44              Belgrano   3.00       nan        nan   
56857 -34.61 -58.42             Caballito   3.00       nan        nan   
56885 -34.63 -58.46                Flores   4.00      3.00        nan   
56895    nan    nan              Barracas   4.00      3.00        nan   
56898 -34.56 -58.44              Belgrano   4.00       nan        nan   
56906 -34.63 -58.45             Caballito   4.00       nan        nan   
56941 -34.57 -58.45              Belgrano   1.00       nan        nan   
56942 -34.57 -58.45              Belgrano   2.00       nan        nan   
56954 -34.62 -58.53            Villa Real   3.00       nan        nan   
57228 -34.62 -58.38             Monserrat   1.00       nan        nan   
57306 -34.62 -58.42               Almagro   2.00      1.00        nan   
57345 -34.60 -58.40              Recoleta   1.00      1.00        nan   
57381 -34.60 -58.45          Villa Crespo   1.00      1.00        nan   
57388 -34.60 -58.43          Villa Crespo   1.00      1.00        nan   
57393 -34.60 -58.45          Villa Crespo   1.00      1.00        nan   
57394 -34.60 -58.45          Villa Crespo   1.00      1.00        nan   
57402 -34.62 -58.36         Puerto Madero   1.00      1.00        nan   
57487 -34.60 -58.45          Villa Crespo   3.00      2.00        n

There are many nan values wich I'll correct with passing a formula based on common sense of how many bathrooms there are in normal appartments.

In [39]:
df['bathrooms'] = df.apply(lambda row: (np.sqrt(row['rooms'])+1).astype(int) if np.isnan(row['bathrooms'])  else row['bathrooms'], axis=1)

There are some outliers where small apartments have too many bathrooms. Same formula will be applyed to cases where there are more bathrooms than rooms.

In [40]:
df['bathrooms'] = df.apply(lambda row: (np.sqrt(row['rooms'])+1).astype(int) if row['bathrooms']>row['rooms']  else row['bathrooms'], axis=1)

In [41]:
df.sort_values(by='bathrooms', ascending=False)

lat    lon                l3  rooms  bedrooms  bathrooms  \
11227 -34.64 -58.53           Liniers  10.00     10.00      10.00   
15251 -34.60 -58.40          Recoleta  15.00       nan       9.00   
15248 -34.61 -58.38         Monserrat   9.00       nan       9.00   
56923 -34.65 -58.49      Villa Lugano   9.00       nan       9.00   
45777 -34.65 -58.47      Villa Lugano   9.00      9.00       9.00   
44515 -34.56 -58.45          Belgrano   7.00      5.00       7.00   
28424 -34.58 -58.40           Palermo   8.00      4.00       7.00   
21726 -34.62 -58.37         San Telmo  15.00     12.00       7.00   
40524    nan    nan          Belgrano   6.00       nan       6.00   
17134    nan    nan      Barrio Norte   6.00      5.00       6.00   
47687 -34.59 -58.38          Recoleta   7.00       nan       6.00   
41482 -34.58 -58.45        Colegiales   8.00       nan       6.00   
42207 -34.61 -58.39       San Nicolás   8.00       nan       6.00   
47719 -34.59 -58.38      Barrio Norte   7.00       nan       6.00   
43306 -34.57 -58.44           Palermo   6.00       nan       6.00   
6123  -34.58 -58.43           Palermo   6.00      4.00       6.00   
4104  -34.57 -58.44          Belgrano   6.00       nan       6.00   
6128  -34.56 -58.45          Belgrano   8.00      4.00       6.00   
49101 -34.58 -58.40           Palermo   8.00      3.00       6.00   
49665 -34.59 -58.38          Recoleta   6.00      4.00       6.00   
17135 -34.59 -58.39          Recoleta   6.00      5.00       6.00   
6812  -34.56 -58.45          Belgrano   6.00       nan       6.00   
4029  -34.61 -58.36     Puerto Madero   6.00      5.00       6.00   
8003  -34.60 -58.40      Barrio Norte  10.00       nan       6.00   
51278 -34.55 -58.44          Belgrano   7.00      5.00       6.00   
1648     nan    nan           Palermo   6.00       nan       6.00   
9725  -34.55 -58.46             Nuñez   6.00       nan       6.00   
9818     nan    nan          Belgrano   7.00       nan       6.00   
9820  -34.55 -58.46          Belgrano   6.00       nan       6.00   
9830  -34.58 -58.45          Belgrano   8.00       nan       6.00   
...      ...    ...               ...    ...       ...        ...   
8981  -34.62 -58.44         Caballito   1.00       nan       1.00   
8983  -34.62 -58.46         Caballito   1.00       nan       1.00   
8985  -34.59 -58.45         Chacarita   1.00       nan       1.00   
29303 -34.65 -58.46            Flores   3.00      2.00       1.00   
29302 -34.63 -58.36              Boca   3.00      2.00       1.00   
8956  -34.61 -58.40          Congreso   3.00       nan       1.00   
8955  -34.56 -58.45          Belgrano   1.00       nan       1.00   
29470 -34.57 -58.43           Palermo   2.00       nan       1.00   
29415 -34.62 -58.40       San Nicolás   1.00       nan       1.00   
29469    nan    nan           Palermo   2.00       nan       1.00   
29467 -34.65 -58.51           Liniers   2.00       nan       1.00   
29466 -34.64 -58.52           Liniers   2.00      1.00       1.00   
29465 -34.56 -58.47           Coghlan   2.00      1.00       1.00   
29463 -34.61 -58.42           Almagro   2.00      1.00       1.00   
29461 -34.55 -58.47             Nuñez   2.00       nan       1.00   
29424 -34.62 -58.43  Parque Chacabuco   1.00       nan       1.00   
29423 -34.57 -58.48     Villa Urquiza   1.00       nan       1.00   
8952  -34.55 -58.45          Belgrano   3.00       nan       1.00   
29405 -34.62 -58.43         Caballito   1.00       nan       1.00   
29353 -34.58 -58.47     Villa Ortuzar   4.00      3.00       1.00   
29404 -34.61 -58.44         Caballito   1.00       nan       1.00   
29403 -34.62 -58.41         Balvanera   1.00       nan       1.00   
29402 -34.54 -58.47          Saavedra   1.00       nan       1.00   
29401 -34.59 -58.39          Recoleta   1.00       nan       1.00   
29400 -34.55 -58.45          Belgrano   1.00       nan       1.00   
29396 -34.58 -58.42           Palermo   1.00       nan       1.00   
29389 -

### Next feature to process is bedrooms. Highly skewed and 42% missing values.

In [42]:
df.sort_values(by='bedrooms', ascending=False)

lat    lon                    l3  rooms  bedrooms  bathrooms  \
35082 -34.62 -58.50          Monte Castro   3.00     21.00       1.00   
21726 -34.62 -58.37             San Telmo  15.00     12.00       7.00   
48304    nan    nan              Belgrano   2.00     12.00       1.00   
38507 -34.61 -58.44             Caballito   2.00     11.00       1.00   
11227 -34.64 -58.53               Liniers  10.00     10.00      10.00   
29212 -34.60 -58.40          Barrio Norte  10.00     10.00       2.00   
45777 -34.65 -58.47          Villa Lugano   9.00      9.00       9.00   
30659 -34.60 -58.39          Barrio Norte  10.00      9.00       3.00   
27527 -34.61 -58.40             Balvanera   9.00      9.00       3.00   
47608    nan    nan  Centro / Microcentro  12.00      8.00       2.00   
8712  -34.60 -58.46              Paternal   9.00      8.00       3.00   
7139  -34.62 -58.37             San Telmo   8.00      8.00       3.00   
24719 -34.59 -58.39              Recoleta  14.00      8.00       4.00   
1968  -34.62 -58.37             San Telmo   8.00      8.00       2.00   
7138  -34.59 -58.44          Villa Crespo   8.00      7.00       1.00   
46165 -34.61 -58.38         San Cristobal   8.00      7.00       2.00   
20108 -34.60 -58.40             Balvanera   7.00      7.00       1.00   
20110 -34.60 -58.39              Recoleta  10.00      7.00       5.00   
4030  -34.62 -58.37             San Telmo   9.00      7.00       3.00   
49169 -34.61 -58.38         San Cristobal   8.00      7.00       2.00   
22932 -34.61 -58.39             Monserrat   8.00      7.00       2.00   
30289 -34.62 -58.39             Balvanera   7.00      6.00       2.00   
23831 -34.59 -58.39          Barrio Norte   7.00      6.00       4.00   
10157 -34.60 -58.39  Centro / Microcentro   7.00      6.00       3.00   
21079 -34.61 -58.40             Balvanera   7.00      6.00       3.00   
57917 -34.58 -58.46            Colegiales   7.00      6.00       5.00   
22928 -34.59 -58.39          Barrio Norte   8.00      6.00       3.00   
53154 -34.60 -58.38  Centro / Microcentro   8.00      6.00       2.00   
31907 -34.58 -58.50      Villa Pueyrredón   8.00      6.00       1.00   
12551 -34.56 -58.45              Belgrano   7.00      6.00       5.00   
...      ...    ...                   ...    ...       ...        ...   
57766 -34.54 -58.46                 Nuñez   3.00       nan       1.00   
57767 -34.61 -58.41               Almagro   3.00       nan       2.00   
57776    nan    nan               Palermo   3.00       nan       1.00   
57778    nan    nan               Palermo   3.00       nan       2.00   
57781 -34.57 -58.43               Palermo   3.00       nan       3.00   
57813 -34.57 -58.45              Belgrano   3.00       nan       2.00   
57820 -34.59 -58.40              Recoleta   3.00       nan       1.00   
57824 -34.62 -58.44             Caballito   3.00       nan       2.00   
57825 -34.61 -58.44             Caballito   3.00       nan       1.00   
57830 -34.62 -58.44             Caballito   3.00       nan       2.00   
57833 -34.62 -58.44             Caballito   3.00       nan       2.00   
57834 -34.62 -58.44             Caballito   3.00       nan       2.00   
57836 -34.59 -58.40          Barrio Norte   3.00       nan       2.00   
57847 -34.60 -58.44          Villa Crespo   3.00       nan       1.00   
57857 -34.64 -58.48     Parque Avellaneda   3.00       nan       1.00   
57858 -34.64 -58.48     Parque Avellaneda   3.00       nan       1.00   
57859 -34.64 -58.48     Parque Avellaneda   3.00       nan       1.00   
57860 -34.64 -58.48     Parque Avellaneda   3.00       nan       1.00   
57867 -34.57 -58.43               Palermo   4.00       nan       2.00   
57868 -34.56 -58.44               Palermo   4.00       nan       1.00   
57874 -34.55 -58.46              Belgrano   4.00       nan       3.00   
57891 -34.62 -58.44             Caballito   4.00       nan       2.00   
57892 -34.57 -58.46            Colegiales   4.00       nan       1.

In [43]:
df['bedrooms'] = df.apply(lambda row: row['rooms'] if (np.isnan(row['bedrooms']))& (row['rooms']==1) else ((row['rooms']-1) if (np.isnan(row['bedrooms']))& (row['rooms']>1) else row['rooms']) , axis=1)

In [44]:
df.sort_values(by='bedrooms', ascending=False)

lat    lon                    l3  rooms  bedrooms  bathrooms  \
43447 -34.63 -58.36                  Boca  18.00     17.00       3.00   
21726 -34.62 -58.37             San Telmo  15.00     15.00       7.00   
15251 -34.60 -58.40              Recoleta  15.00     14.00       9.00   
24719 -34.59 -58.39              Recoleta  14.00     14.00       4.00   
47607 -34.60 -58.38              Recoleta  12.00     12.00       3.00   
24718 -34.59 -58.40              Recoleta  12.00     12.00       4.00   
26811 -34.59 -58.39              Recoleta  12.00     12.00       3.00   
6657  -34.62 -58.37             San Telmo  12.00     12.00       3.00   
34053 -34.60 -58.38                Retiro  12.00     12.00       3.00   
34057 -34.59 -58.40              Recoleta  12.00     12.00       3.00   
47608    nan    nan  Centro / Microcentro  12.00     12.00       2.00   
20662 -34.60 -58.38                Retiro  11.00     11.00       1.00   
21725 -34.60 -58.38           San Nicolás  11.00     11.00       2.00   
40610 -34.59 -58.39              Recoleta  11.00     10.00       4.00   
29212 -34.60 -58.40          Barrio Norte  10.00     10.00       2.00   
8757  -34.62 -58.37             San Telmo  11.00     10.00       5.00   
31515 -34.57 -58.45              Belgrano  10.00     10.00       4.00   
11227 -34.64 -58.53               Liniers  10.00     10.00      10.00   
20110 -34.60 -58.39              Recoleta  10.00     10.00       5.00   
26810 -34.59 -58.39              Recoleta  10.00     10.00       3.00   
15250 -34.59 -58.38                Retiro  10.00     10.00       3.00   
35738 -34.62 -58.38             San Telmo  11.00     10.00       4.00   
9906  -34.61 -58.48      Villa Santa Rita  11.00     10.00       1.00   
30659 -34.60 -58.39          Barrio Norte  10.00     10.00       3.00   
51272 -34.60 -58.40          Barrio Norte  10.00     10.00       5.00   
44903    nan    nan              Recoleta  10.00     10.00       3.00   
379   -34.59 -58.40              Recoleta  10.00     10.00       4.00   
7997  -34.60 -58.40          Barrio Norte  11.00     10.00       4.00   
55627 -34.59 -58.39              Recoleta   9.00      9.00       3.00   
51262 -34.58 -58.40               Palermo   9.00      9.00       3.00   
...      ...    ...                   ...    ...       ...        ...   
35261 -34.60 -58.41              Recoleta   1.00      1.00       1.00   
35229 -34.62 -58.42                 Boedo   1.00      1.00       1.00   
35244 -34.58 -58.43               Palermo   1.00      1.00       1.00   
35230 -34.60 -58.41                Abasto   1.00      1.00       1.00   
35231 -34.61 -58.50                Abasto   1.00      1.00       1.00   
35232 -34.60 -58.41                Abasto   1.00      1.00       1.00   
35234 -34.60 -58.37                Retiro   1.00      1.00       1.00   
35235 -34.60 -58.43               Almagro   1.00      1.00       1.00   
35236 -34.60 -58.41               Almagro   1.00      1.00       1.00   
35238    nan    nan               Palermo   1.00      1.00       1.00   
35239 -34.60 -58.42               Palermo   1.00      1.00       1.00   
35240 -34.58 -58.42               Palermo   1.00      1.00       1.00   
35241 -34.59 -58.44               Palermo   1.00      1.00       1.00   
35242 -34.59 -58.42               Palermo   1.00      1.00       1.00   
35243 -34.60 -58.42               Palermo   1.00      1.00       1.00   
35245    nan    nan               Palermo   1.00      1.00       1.00   
35259    nan    nan              Recoleta   1.00      1.00       1.00   
35246    nan    nan               Palermo   1.00      1.00       1.00   
35247 -34.58 -58.42               Palermo   1.00      1.00       1.00   
35248 -34.59 -58.43               Palermo   1.00      1.00       1.00   
35249    nan    nan               Palermo   1.00      1.00       1.00   
35250    nan    nan               Palermo   1.00      1.00       1.00   
35251 -34.55 -58.44              Belgrano   1.00      1.00       1.